In [1]:
# ============================================================
# 🧠 Train Gating Network for Sparse MoE (v3)
#  - Encoder: google/embeddinggemma-300m (SentenceTransformer)
#  - Gating: 2-layer MLP trên sentence embedding
#  - Mục tiêu: chọn expert phù hợp cho mỗi câu hỏi
# ============================================================

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ------------------------------------------------------------
# ⚙️ CONFIG
# ------------------------------------------------------------
DATA_FILE = "gating_train.csv"              # merged dataset cho tất cả expert
MODEL_NAME = "google/embeddinggemma-300m"   # cùng encoder với Dual Expert
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
EPOCHS = 10
LR = 2e-5
SEED = 42

SAVE_DIR = "models_gating_v3"
os.makedirs(SAVE_DIR, exist_ok=True)

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ------------------------------------------------------------
# 🧩 Dataset
#   - question: str
#   - expert_label: tên expert (Advice / Treatment / Disease / Topic / ...)
# ------------------------------------------------------------
class GatingDataset(Dataset):
    def __init__(self, df, label2id):
        self.texts = df["question"].astype(str).tolist()
        self.labels = df["expert_label"].astype(str).tolist()
        self.label2id = label2id

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label_name = self.labels[idx]

        # one-hot multi-label vector (num_experts,)
        y = torch.zeros(len(self.label2id), dtype=torch.float32)
        if label_name in self.label2id:
            y[self.label2id[label_name]] = 1.0
        return text, y


# ------------------------------------------------------------
# 🔧 Helper encode_texts (giống dual expert v3)
# ------------------------------------------------------------
def encode_texts(encoder: SentenceTransformer, texts, batch_size=64):
    with torch.no_grad():
        emb = encoder.encode(
            texts,
            batch_size=batch_size,
            convert_to_tensor=True,
            device=DEVICE,
            show_progress_bar=False
        )
    return emb  # (N, D)


# ------------------------------------------------------------
# 🧠 Gating MLP
#   - Input: sentence embedding (D)
#   - Output: logits (num_experts) — dùng BCEWithLogits cho linh hoạt multi-label
# ------------------------------------------------------------
class GatingMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=512):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h)  # (B, num_classes)


# ------------------------------------------------------------
# 🚀 Train Gating
# ------------------------------------------------------------
def train_gating():
    print("\n=== 🧩 Training Gating Network for Sparse MoE (v3) ===")

    # 1) Load data
    df = pd.read_csv(DATA_FILE)
    df = df.dropna(subset=["question", "expert_label"])

    # Map expert_label -> id
    experts = sorted(df["expert_label"].astype(str).unique())
    label2id = {label: i for i, label in enumerate(experts)}
    id2label = {i: label for label, i in label2id.items()}

    print(f"✅ Loaded {len(df)} samples")
    print(f"✅ Num experts: {len(experts)}")
    print("   Experts:", experts)

    dataset = GatingDataset(df, label2id)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    # 2) Base encoder (frozen)
    base_encoder = SentenceTransformer(MODEL_NAME)
    base_encoder.to(DEVICE)
    base_encoder.eval()
    for p in base_encoder.parameters():
        p.requires_grad = False

    # Lấy dim embedding
    try:
        input_dim = base_encoder.get_sentence_embedding_dimension()
    except Exception:
        test_emb = encode_texts(base_encoder, ["test"])
        input_dim = test_emb.size(1)

    # 3) Gating model
    num_classes = len(label2id)
    model = GatingMLP(input_dim=input_dim, num_classes=num_classes).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.BCEWithLogitsLoss()

    use_amp = DEVICE == "cuda"
    scaler = torch.amp.GradScaler("cuda") if use_amp else None

    # 4) Training loop
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0.0
        total_acc = 0.0
        n_steps = 0

        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

        for batch in pbar:
            texts, y = batch
            y = y.to(DEVICE)

            if use_amp:
                ctx_mgr = torch.amp.autocast("cuda")
            else:
                from contextlib import nullcontext
                ctx_mgr = nullcontext()

            with ctx_mgr:
                # Encode texts -> embeddings
                emb = encode_texts(base_encoder, list(texts))  # (B, D)

                # Forward gating
                logits = model(emb)  # (B, num_classes)
                loss = loss_fn(logits, y)

                # metric: top-1 accuracy (argmax)
                with torch.no_grad():
                    preds = torch.argmax(logits, dim=1)
                    gold = torch.argmax(y, dim=1)
                    acc = (preds == gold).float().mean().item()

            optimizer.zero_grad()
            if use_amp:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            n_steps += 1
            total_loss += loss.item()
            total_acc += acc

            pbar.set_postfix(
                loss=f"{total_loss/n_steps:.4f}",
                acc=f"{total_acc/n_steps:.4f}"
            )

        print(
            f"✅ Epoch {epoch+1} | "
            f"Loss={total_loss/n_steps:.4f} | "
            f"Top1 Acc={total_acc/n_steps:.4f}"
        )

    # 5) Save model + label mapping
    torch.save(model.state_dict(), os.path.join(SAVE_DIR, "gating_mlp.pt"))
    torch.save(
        {
            "label2id": label2id,
            "id2label": id2label,
            "input_dim": input_dim,
            "num_classes": num_classes,
            "model_name": MODEL_NAME,
        },
        os.path.join(SAVE_DIR, "gating_meta.pt"),
    )

    print(f"\n💾 Saved gating model to: {SAVE_DIR}")


# ------------------------------------------------------------
if __name__ == "__main__":
    train_gating()


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



=== 🧩 Training Gating Network for Sparse MoE (v3) ===
✅ Loaded 227322 samples
✅ Num experts: 12
   Experts: ['Advice', 'Application', 'Cause', 'Complication', 'Definition', 'Detail', 'Population', 'Prevention', 'RiskFactor', 'SubDisease', 'Symptom', 'Treatment']


Epoch 1/10: 100%|██████████| 7104/7104 [03:24<00:00, 34.78it/s, acc=0.7500, loss=0.2302]


✅ Epoch 1 | Loss=0.2302 | Top1 Acc=0.7500


Epoch 2/10: 100%|██████████| 7104/7104 [03:22<00:00, 35.10it/s, acc=0.9575, loss=0.0612]


...

✅ Epoch 8 | Loss=0.0011 | Top1 Acc=0.9997


Epoch 9/10: 100%|██████████| 7104/7104 [03:25<00:00, 34.54it/s, acc=0.9998, loss=0.0008]


✅ Epoch 9 | Loss=0.0008 | Top1 Acc=0.9998


Epoch 10/10: 100%|██████████| 7104/7104 [03:23<00:00, 34.95it/s, acc=0.9998, loss=0.0006]


✅ Epoch 10 | Loss=0.0006 | Top1 Acc=0.9998

💾 Saved gating model to: models_gating_v3


In [14]:
import os
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer


META_PATH = os.path.join(SAVE_DIR, "gating_meta.pt")
STATE_PATH = os.path.join(SAVE_DIR, "gating_mlp.pt")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
class GatingMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=512):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h)  # (B, num_classes)


def encode_texts(encoder: SentenceTransformer, texts, batch_size=64):
    with torch.no_grad():
        emb = encoder.encode(
            texts,
            batch_size=batch_size,
            convert_to_tensor=True,
            device=DEVICE,
            show_progress_bar=False
        )
    return emb  # (N, D)



meta = torch.load(META_PATH, map_location=DEVICE)
label2id = meta["label2id"]
id2label = meta["id2label"]
input_dim = meta["input_dim"]
num_classes = meta["num_classes"]
model_name = meta["model_name"]

encoder = SentenceTransformer(model_name).to(DEVICE)
encoder.eval()
for p in encoder.parameters():
    p.requires_grad = False

model = GatingMLP(input_dim=input_dim, num_classes=num_classes).to(DEVICE)
model.load_state_dict(torch.load(STATE_PATH, map_location=DEVICE))
model.eval()

QUESTION = "hiv là gì và được điều trị như thế nào"

with torch.no_grad():
    emb = encode_texts(encoder, [QUESTION])  # (1, D)
    logits = model(emb)                      # (1, num_classes)
    probs = torch.sigmoid(logits).squeeze(0) # (num_classes,)

    topk = torch.topk(probs, k=5)
    print(f"\n🧪 Question: {QUESTION}")
    print("Top experts:")
    for score, idx in zip(topk.values, topk.indices):
        name = id2label[int(idx)]
        print(f"  {name:15s} → {float(score):.4f}")



🧪 Question: hiv là gì và được điều trị như thế nào
Top experts:
  Treatment       → 0.9490
  Definition      → 0.6008
  Detail          → 0.0078
  Application     → 0.0000
  Advice          → 0.0000


In [17]:
import os
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import torch.nn as nn

META_PATH = os.path.join(SAVE_DIR, "gating_meta.pt")
STATE_PATH = os.path.join(SAVE_DIR, "gating_mlp.pt")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# -------------------------
# Gating MLP
# -------------------------
class GatingMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=512):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h)


# -------------------------
# Load metadata
# -------------------------
meta = torch.load(META_PATH, map_location=DEVICE)
label2id = meta["label2id"]
id2label = meta["id2label"]
input_dim = meta["input_dim"]
num_classes = meta["num_classes"]
model_name = meta["model_name"]

# -------------------------
# Load encoder (FAST MODE)
# -------------------------
encoder = SentenceTransformer(model_name, device=DEVICE)
encoder.eval()
for p in encoder.parameters():
    p.requires_grad = False


# FAST INFERENCE: encode 1 câu bằng .encode() nhưng tránh sync
def fast_encode(text):
    with torch.inference_mode():
        emb = encoder.encode(
            [text],
            batch_size=1,
            convert_to_tensor=True,
            device=DEVICE,
            show_progress_bar=False
        )
    return emb


# -------------------------
# Load gating model
# -------------------------
model = GatingMLP(input_dim, num_classes).to(DEVICE)
model.load_state_dict(torch.load(STATE_PATH, map_location=DEVICE))
model.eval()

# -------------------------
# Test
# -------------------------
QUESTION = "hiv là gì"

with torch.inference_mode():
    emb = fast_encode(QUESTION)
    logits = model(emb)
    probs = torch.sigmoid(logits).squeeze(0)

    topk = torch.topk(probs, k=5)

    print(f"\n🧪 Question: {QUESTION}")
    print("Top experts:")
    for score, idx in zip(topk.values, topk.indices):
        print(f"{id2label[int(idx)]:20s} → {float(score):.4f}")



🧪 Question: hiv là gì
Top experts:
Definition           → 0.9999
Detail               → 0.0004
SubDisease           → 0.0000
Complication         → 0.0000
Application          → 0.0000


cuda:0
